# Explore metabolism of cancer cells
Corresponds to fig 3 and sfig 3 in draft.

In [ ]:
import os
import numpy as np
import anndata as ad
import pandas as pd
import scanpy as sc
from plotnine import *

In [ ]:
adata = ad.read_h5ad('../../data/adata_consensus_cell_types.h5ad')

In [ ]:
all_functional_and_metab = ['CA9', 'CD98', 'CytC', 'MSH2', 'MCT1', 'ASCT2',
       'LDH', 'STING1', 'GS', 'GLS', 'ATP5A', 'CS', 'PKM2', 'GLUT1', 'MSH6', 'ARG1', 'CPT1A', 'Ki67']

In [ ]:
# Group rows by cell type and compute median expression
df = adata.obs.loc[:,all_functional_and_metab]
df["cell_type"] = adata.obs["annotation_consensus"].values
df = df.groupby("cell_type").median().T
df = df.drop("Unclear", axis=1)

In [ ]:
from scipy.cluster.hierarchy import linkage, leaves_list
# Perform hierarchical clustering
row_linkage = linkage(df, method='complete')
col_linkage = linkage(df.T, method='complete')

# Get the order of rows and columns
row_order = leaves_list(row_linkage)
col_order = leaves_list(col_linkage)

# Scale df values so that each row (abundance for a given marker) is between 0 and 1
df = df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)), axis = 1)

# Reorder the DataFrame
df_reordered = df.iloc[row_order, :].iloc[:, col_order]

# Reorder the DataFrame
df_reordered = df.iloc[row_order, :].iloc[:, col_order]

# Melt the DataFrame for plotnine
df_melted = df_reordered.melt(ignore_index=False).reset_index()

# Update the levels of the categorical variables to reflect the new order
df_melted["index"] = pd.Categorical(df_melted["index"], categories=df_reordered.index, ordered=True)
df_melted["cell_type"] = pd.Categorical(df_melted["cell_type"], categories=df_reordered.columns.to_list(), ordered=True)

In [ ]:
# Display as heatmap
gp = (
    ggplot(df_melted, aes(x="index", y="cell_type")) 
    + geom_tile(aes(fill="value")) 
    + theme_classic() 
    + theme(axis_text_x=element_text(angle=90)) 
    + labs(y="Cell type", x="Marker", fill="Median\nabundance") 
    # + coord_equal()
    # Use reverted plasma color palette
    + scale_fill_gradientn(colors = ["#EFF822", "#CC4977","#0F0782"])
)
ggsave(gp, "../../figures/fig3/heatmap_all_markers_scaled.png", width = 6, height = 3.2)
ggsave(gp, "../../figures/fig3/heatmap_all_markers_scaled.pdf", width = 6, height = 3.2)
gp

In [ ]:
# Repeat the analysis for each tumor stage
clini = pd.read_csv("../../data/summary_clinical_data_modified.csv", index_col=2)
adata.obs = adata.obs.merge(clini, left_on="fov", right_index=True, how="left")

In [ ]:
adata.obs["Stage"] = adata.obs["pT group"]
# # E1 and E2 samples are annotated 'SCT' while E3 and E4 are 'Colon-no.'
adata.obs.loc[adata.obs.fov.str.contains("E4"), "Stage"] = "Colon-no."
adata.obs.loc[adata.obs.fov.str.contains("E3"), "Stage"] = "Colon-no."
adata.obs.loc[adata.obs.fov.str.contains("E2"), "Stage"] = "SCT"
adata.obs.loc[adata.obs.fov.str.contains("E1"), "Stage"] = "SCT"

In [ ]:
# Export for later spatial analyses
adata.obs.to_csv("../../data/cell_table_with_types_stage.csv")

In [ ]:
for stage in adata.obs['Stage'].unique():
    if pd.isna(stage):
        continue

    df = adata.obs.loc[adata.obs['Stage'] == stage, all_functional_and_metab]
    df["cell_type"] = adata.obs.loc[adata.obs['Stage'] == stage, "annotation_consensus"].values
    df = df.groupby("cell_type").median().T
    df = df.drop("Unclear", axis=1)

    # We keep the order of the markers and cell types from the previous analysis for consistency

    # Scale df values so that each row (abundance for a given marker) is between 0 and 1
    df = df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)), axis = 1)

    # Reorder the DataFrame
    df_reordered = df.iloc[row_order, :].iloc[:, col_order]

    # Melt the DataFrame for plotnine
    df_melted = df_reordered.melt(ignore_index=False).reset_index()

    # Update the levels of the categorical variables to reflect the new order
    df_melted["index"] = pd.Categorical(df_melted["index"], categories=df_reordered.index, ordered=True)
    df_melted["cell_type"] = pd.Categorical(df_melted["cell_type"], categories=df_reordered.columns.to_list(), ordered=True)

    # Display as heatmap
    gp = (
        ggplot(df_melted, aes(x="index", y="cell_type")) 
        + geom_tile(aes(fill="value")) 
        + theme_classic() 
        + theme(axis_text_x=element_text(angle=90)) 
        + labs(y="Cell type", x="Marker", fill="Median\nabundance") 
        # Use reverted plasma color palette
        + scale_fill_gradientn(colors = ["#EFF822", "#CC4977","#0F0782"])
        + ggtitle(f"Stage {stage}")
    )
    print(gp)
    ggsave(gp, f"../../figures/fig3/heatmap_all_markers_scaled_stage_{stage}.png", width = 6, height = 3.2)
    ggsave(gp, f"../../figures/fig3/heatmap_all_markers_scaled_stage_{stage}.pdf", width = 6, height = 3.2)

GLUT1 high in neutrophils and cancer cells – not in healthy epithelial cells. Also higher proliferation (Ki67). Also CD98 (neutral aa transporter) high, MSH6 / MSH2 / LDH low in cancer vs epithelial.  
Caveat: scaled scores are computed per marker per stage and not across stages. Scaling values before computing the median means giving weight to outliers (if using min and max) or doing what's done already (if using small/large percentiles).

## Cancer cell metab subgroups
Leiden on metabolic markers only? Enrichment by pathway then cluster?

### Score 0: metabolic marker intensities

In [ ]:
df = adata.obs.loc[adata.obs.consensus == "Cancer_cell",["LDH", "GLUT1",'pT group']] 
df["Stage"] = df["pT group"].replace({pd.NA: "pT0"})
gp = (ggplot(df, aes(x="LDH", y="GLUT1", color='Stage')) 
      + geom_point(alpha = 0.1) + theme_classic() + facet_wrap("~Stage")
)
gp

In [ ]:
df = adata.obs.loc[adata.obs.consensus == "Cancer_cell",["CD98", "Ki67",'pT group']] 
df["Stage"] = df["pT group"].replace({pd.NA: "pT0"})
gp = (ggplot(df, aes(x="CD98", y="Ki67", color='Stage')) 
      + geom_point(alpha = 0.1) + theme_classic() + facet_wrap("~Stage")
)
gp

In [ ]:
df = adata.obs.loc[adata.obs.fov == "A1c",adata.obs.columns[:23]]
df["cell_type"] = adata.obs.loc[adata.obs.fov == "A1c", "annotation_consensus"].values
df = df.groupby("cell_type").median().T
df = df.drop("Unclear", axis=1)

df

In [ ]:
df = adata.obs.loc[adata.obs.fov == "E4a",adata.obs.columns[:23]]
df["cell_type"] = adata.obs.loc[adata.obs.fov == "E4a", "annotation_consensus"].values
df = df.groupby("cell_type").median().T
df = df.drop("Unclear", axis=1)

df

In [ ]:
df = pd.DataFrame(adata.X[adata.obs.fov == "E4a",:], columns=adata.var_names)
df["cell_type"] = adata.obs.loc[adata.obs.fov == "E4a", "annotation_consensus"].values
df = df.groupby("cell_type").median().T
df = df.drop("Unclear", axis=1)

df

In [ ]:
df = adata.obs.loc[adata.obs.consensus == "Cancer_cell",["LDH", "GLUT1",'pT group','fov']] 
df["Stage"] = df["pT group"]
# # E1 and E2 samples are annotated 'SCT' while E3 and E4 are 'Colon-no.'
df.loc[df.fov.str.contains("E4"), "Stage"] = "Colon-no."
df.loc[df.fov.str.contains("E3"), "Stage"] = "Colon-no."
df.loc[df.fov.str.contains("E2"), "Stage"] = "SCT"
df.loc[df.fov.str.contains("E1"), "Stage"] = "SCT"

gp = (ggplot(df, aes(x="LDH", y="GLUT1", color='Stage')) 
      + geom_point(alpha = 0.1) + theme_classic() + facet_wrap("~Stage")
)
gp

In [ ]:
df = adata.obs.loc[adata.obs.consensus == "Cancer_cell",["CD98", "Ki67",'pT group','fov']] 
df["Stage"] = df["pT group"]
# # E1 and E2 samples are annotated 'SCT' while E3 and E4 are 'Colon-no.'
df.loc[df.fov.str.contains("E4"), "Stage"] = "Colon-no."
df.loc[df.fov.str.contains("E3"), "Stage"] = "Colon-no."
df.loc[df.fov.str.contains("E2"), "Stage"] = "SCT"
df.loc[df.fov.str.contains("E1"), "Stage"] = "SCT"

gp = (ggplot(df, aes(x="CD98", y="Ki67", color='Stage')) 
      + geom_point(alpha = 0.1) + theme_classic() + facet_wrap("~Stage")
)
gp

In [ ]:
for stage in adata.obs['pT group'].unique():
    if pd.isna(stage):
        df = adata.obs.loc[adata.obs['pT group'].isna(), all_functional_and_metab]
        df["cell_type"] = adata.obs.loc[adata.obs['pT group'].isna(), "annotation_consensus"].values
    else:
        df = adata.obs.loc[adata.obs['pT group'] == stage, all_functional_and_metab]
        df["cell_type"] = adata.obs.loc[adata.obs['pT group'] == stage, "annotation_consensus"].values
    df = df.groupby("cell_type").median().T
    df = df.drop("Unclear", axis=1)

    # We keep the order of the markers and cell types from the previous analysis for consistency

    # Scale df values so that each row (abundance for a given marker) is between 0 and 1
    df = df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)), axis = 1)

    # Reorder the DataFrame
    df_reordered = df.iloc[row_order, :].iloc[:, col_order]

    # Melt the DataFrame for plotnine
    df_melted = df_reordered.melt(ignore_index=False).reset_index()

    # Update the levels of the categorical variables to reflect the new order
    df_melted["index"] = pd.Categorical(df_melted["index"], categories=df_reordered.index, ordered=True)
    df_melted["cell_type"] = pd.Categorical(df_melted["cell_type"], categories=df_reordered.columns.to_list(), ordered=True)

    # Display as heatmap
    gp = (
        ggplot(df_melted, aes(x="index", y="cell_type")) 
        + geom_tile(aes(fill="value")) 
        + theme_classic() 
        + theme(axis_text_x=element_text(angle=90)) 
        + labs(y="Cell type", x="Marker", fill="Median\nabundance") 
        # Use reverted plasma color palette
        + scale_fill_gradientn(colors = ["#EFF822", "#CC4977","#0F0782"])
        + ggtitle(f"Stage {stage}")
    )
    print(gp)
    # ggsave(gp, f"figures/fig3/heatmap_all_markers_scaled_stage_{stage}.png", width = 6, height = 3.2)

In [ ]:
df_melted

## Cancer cell metabolism
In this section, we explore the variations in metabolic activitiy of epithelial / malignant cells, and how it evolves with the disease stage.  
For now, we ignore the influence of other cell types, and start by looking at ...  
While a classification at the cell level shows the informativeness of individual cell metabolic profiles, not all cells have to be representative of the disease as a whole. The most relevant representation should thus be the one best representing FOVs/samples. For given markers, the aggregation can be attempted as follows: no aggregation, mean profile (train on cells or on mean profiles), median profile (train on cells on or median profiles), clustering and proportions. We want to compare these methods (and ideally later use the same split for multicellular / spatial analyses) -> 4xCV + validation set (per FOV) = 1 healthy donor per fold.  
How does it relate to proliferation/aggressiveness?  
QC: cosine similarity between cells from the same stage?  

### Score 1: keep all markers separated

In [ ]:
metab_markers = ['CA9', 'CD98', 'CytC', 'MCT1', 'ASCT2', 'LDH', 'GS', 'GLS', 'ATP5A', 'CS', 'PKM2', 'GLUT1', 'ARG1', 'CPT1A', 'Ki67']

# Only metabolic markers for cancer/epithelial cells
df = adata.obs.loc[adata.obs.consensus == "Cancer_cell",metab_markers] 
meta = adata.obs.loc[adata.obs.consensus == "Cancer_cell",["Stage","fov"]]

# Only keep well-annotated stages
epithelial_subset = meta["Stage"].isin(["Colon-no.", "pT1", "pT2", "pT3", "pT4"]).values

For visualization:
```python
import umap

# Reduce to 2 dimensions with UMAP
umap_metab = umap.UMAP(random_state=42).fit_transform(df)

umap_metab = pd.DataFrame(umap_metab, columns=["UMAP1", "UMAP2"])
umap_metab["stage"] = adata.obs.loc[adata.obs.consensus == "Cancer_cell",'Stage'].values

gp = (ggplot(umap_metab, aes(x="UMAP1", y="UMAP2", color="stage")) 
      + geom_point() + theme_classic()
)
gp

gp = (ggplot(umap_metab, aes(x="UMAP1", y="UMAP2")) 
      + geom_bin2d() + theme_classic()
      + facet_wrap("~stage")
)
gp 
```

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedGroupKFold, StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Define which fovs will be held out for validation (outer loop)
fov_stage_table = meta.loc[epithelial_subset].drop_duplicates().reset_index(drop=True)
fov_inner, fov_val, y_inner, y_val = train_test_split(
    fov_stage_table["fov"], fov_stage_table["Stage"], test_size=0.2, random_state=0, stratify=fov_stage_table["Stage"])
meta["inner"] = meta["fov"].isin(fov_inner)

In [ ]:
n_splits = 4

cv_folds = StratifiedGroupKFold(n_splits=n_splits)
for train, test in cv_folds.split(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]],
                                  meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"],
                                  groups=meta.loc[epithelial_subset].loc[meta["inner"]]["fov"]):
    # All stages should be present in both train and test
    assert len(meta.loc[epithelial_subset].loc[meta["inner"]].iloc[test].groupby("fov")["Stage"].first().unique()) == 5
    assert len(meta.loc[epithelial_subset].loc[meta["inner"]].iloc[train].groupby("fov")["Stage"].first().unique()) == 5

In [ ]:
cross_val_score(
    XGBClassifier(
        n_estimators=250, 
        max_depth=3, 
        device="cuda", 
        random_state=0),
    df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]],
    LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"]),
    groups=meta.loc[epithelial_subset].loc[meta["inner"]]["fov"], 
    cv=cv_folds, 
    scoring='f1_macro')

In [ ]:
df_per_fov = df.copy()
df_per_fov["fov"] = meta["fov"] 
df_per_fov = df_per_fov.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].groupby("fov").mean()

In [ ]:
# Train on FOV mean, test on FOV mean
cv_folds_fov = StratifiedKFold(n_splits=n_splits)
for train, test in cv_folds_fov.split(df_per_fov,
                                  meta.loc[epithelial_subset].loc[meta["inner"]].groupby("fov").first()["Stage"]
                                  ):
    print(meta.loc[epithelial_subset].loc[meta["inner"]].groupby("fov").first().iloc[test]["Stage"].value_counts())

In [ ]:
cross_val_score(
    XGBClassifier(
        n_estimators=250, 
        max_depth=3, 
        device="cuda", 
        random_state=0),
    df_per_fov,
    LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]].groupby("fov").first()["Stage"]),
    cv=cv_folds_fov, 
    scoring='f1_macro')

In [ ]:
for train, test in cv_folds.split(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]],
                                  meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"],
                                  groups=meta.loc[epithelial_subset].loc[meta["inner"]]["fov"]):
    xgb = XGBClassifier(
        n_estimators=250, 
        max_depth=3, 
        device="cuda", 
        random_state=0)
    xgb.fit(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].iloc[train],
            LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"].iloc[train]))
    # First show f1_score on test data
    preds = xgb.predict(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].iloc[test])
    print(f1_score(LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"].iloc[test]), 
                   preds, 
                   average="macro"))
    # This recapitulates the results of calling `cross_val_score`

    # Train on individual cells, test on FOV mean
    test_fov = meta.loc[epithelial_subset].loc[meta["inner"]]["fov"].iloc[test].unique()
    preds = xgb.predict(df_per_fov.loc[test_fov])
    print(f1_score(LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]].groupby("fov").first()["Stage"].loc[test_fov]), 
                   preds, 
                   average="macro"))

In [ ]:
# Repeat with median instead of mean
df_per_fov = df.copy()
df_per_fov["fov"] = meta["fov"] 
df_per_fov = df_per_fov.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].groupby("fov").median()

In [ ]:
cross_val_score(
    XGBClassifier(
        n_estimators=250, 
        max_depth=3, 
        device="cuda", 
        random_state=0),
    df_per_fov,
    LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]].groupby("fov").first()["Stage"]),
    cv=cv_folds_fov, 
    scoring='f1_macro')

In [ ]:
for train, test in cv_folds.split(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]],
                                  meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"],
                                  groups=meta.loc[epithelial_subset].loc[meta["inner"]]["fov"]):
    xgb = XGBClassifier(
        n_estimators=250, 
        max_depth=3, 
        device="cuda", 
        random_state=0)
    xgb.fit(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].iloc[train],
            LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"].iloc[train]))
    # First show f1_score on test data
    preds = xgb.predict(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].iloc[test])
    print(f1_score(LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"].iloc[test]), 
                   preds, 
                   average="macro"))
    # This recapitulates the results of calling `cross_val_score`

    # Train on individual cells, test on FOV median
    test_fov = meta.loc[epithelial_subset].loc[meta["inner"]]["fov"].iloc[test].unique()
    preds = xgb.predict(df_per_fov.loc[test_fov])
    print(f1_score(LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]].groupby("fov").first()["Stage"].loc[test_fov]), 
                   preds, 
                   average="macro"))

In [ ]:
# Ind
(0.21334178256892686 + 0.3047374279671758 + 0.29207851933589885 + 0.2485800697783061)/4

In [ ]:
# Med
np.mean([0.20805219, 0.19189579, 0.21131957, 0.1926093 ])

In [ ]:
# Ind + Mean pred
(0.18001782001782002+0.4324540247211335+0.36690958164642373+0.16494823006450915)/4

In [ ]:
# Ind + Median pred
(0.16813147914032872 + 0.3699462365591398 + 0.3417610522469857 + 0.18623318819545237)/4

In [ ]:
# Filter based on what methods are kept
# from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
import scanpy as sc
from joblib import Parallel, delayed
import warnings, logging

In [ ]:
def process_fold(train, test, n_neighbors = 30, resolution = 0.5, n_estimators = 250):
    logging.getLogger('tensorflow').setLevel(logging.ERROR)
    with warnings.catch_warnings(action="ignore"):
        # Step 1: Define metabolic clusters on training data
        ad = sc.AnnData(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].iloc[train])
        sc.pp.neighbors(ad, n_neighbors=n_neighbors)
        sc.tl.leiden(ad, resolution=resolution)
        ad.obs.leiden = ad.obs.leiden.values.astype(int)

        # Step 2: Define a classifier to propagate the clusters to the test data
        neigh = KNeighborsClassifier(n_neighbors=n_neighbors, n_jobs=-1)
        neigh.fit(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].iloc[train].values, 
                ad.obs.leiden.values)

        # Step 3: Compute proportion of cells in each cluster for each FOV in the training data
        train_fov_cluster_composition = pd.DataFrame(ad.obs.leiden.values, columns=["Cluster"])
        train_fov_cluster_composition["fov"] = (
            meta.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].iloc[train]["fov"].values
        )
        train_fov_cluster_composition = (
            train_fov_cluster_composition.groupby("fov")["Cluster"].value_counts().unstack().fillna(0)
        )
        # Normalize by the number of cells in each FOV
        train_fov_cluster_composition = (
            train_fov_cluster_composition.div(train_fov_cluster_composition.sum(axis=1), axis=0)
        )

        # Step 4: Train a classifier on the training data composition to predict the stage of each FOV
        xgb = XGBClassifier(
            n_estimators=n_estimators, 
            max_depth=3, 
            device="cuda", 
            random_state=0)
        xgb.fit(train_fov_cluster_composition,
                LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]].groupby("fov").first()["Stage"].loc[train_fov_cluster_composition.index]))
        
        # Step 5: Predict clusters on test data
        test_clusters = neigh.predict(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].iloc[test].values)

        # Step 6: Compute proportion of cells in each cluster for each FOV in the test data
        test_fov_cluster_composition = pd.DataFrame(test_clusters, columns=["Cluster"])
        test_fov_cluster_composition["fov"] = meta.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].iloc[test]["fov"].values
        test_fov_cluster_composition = test_fov_cluster_composition.groupby("fov")["Cluster"].value_counts().unstack().fillna(0)
        # Normalize by the number of cells in each FOV
        test_fov_cluster_composition = test_fov_cluster_composition.div(test_fov_cluster_composition.sum(axis=1), axis=0)
        
        # Ensure all clusters are covered in the testing dataframe
        for cluster in train_fov_cluster_composition.columns:
            if cluster not in test_fov_cluster_composition.columns:
                test_fov_cluster_composition[cluster] = 0

        # Reorder columns to match the training set
        test_fov_cluster_composition = test_fov_cluster_composition[train_fov_cluster_composition.columns]

        # Step 7: Predict stage of each FOV in the test data
        preds = xgb.predict(test_fov_cluster_composition)

        # Step 8: Compute f1_score
        score = f1_score(LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]].groupby("fov").first()["Stage"].loc[test_fov_cluster_composition.index]), 
                        preds, 
                        average="macro")
        return score

In [ ]:
# Function to process a single set of hyperparameters
def process_hyperparameters(r_resolution, r_neighbors, r_estimators):
    r_neighbors = int(r_neighbors)
    print(r_resolution, r_neighbors, r_estimators)
    scores = Parallel(n_jobs=1)(delayed(process_fold)(train, 
                                                       test, 
                                                       n_neighbors=r_neighbors,
                                                       resolution=r_resolution,
                                                       n_estimators=r_estimators) 
                                 for train, test in cv_folds.split(
        df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]],
        meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"],
        groups=meta.loc[epithelial_subset].loc[meta["inner"]]["fov"]
    ))
    mean_score = np.mean(scores)
    print("[Results]  ", r_resolution, r_neighbors, r_estimators, mean_score)
    
    # Ensure file exists
    if not os.path.isfile("../../data/cluster_f1_scores.txt"):
        with open("../../data/cluster_f1_scores.txt", "w") as f:
            f.write("resolution,neighbors,estimators,score\n")

    # Append to f1 score log file
    with open(f"../../data/cluster_f1_scores.txt", "a") as f:
        f.write(f"{r_resolution},{r_neighbors},{r_estimators},{mean_score}\n")

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# Perform nb_sim tests for random values of n_neighbors, RF estimators and Leiden resolution
nb_sim = 10
np.random.seed(1)
resolutions = np.round(np.random.rand(nb_sim), 2)
neighbors = np.ceil(100 * np.random.pareto(6, nb_sim))
estimators = np.random.randint(500, size=nb_sim)

# Parallelize the hyperparameter loop
Parallel(n_jobs=28)(delayed(process_hyperparameters)(r_resolution, r_neighbors, r_estimators)
                for r_resolution, r_neighbors, r_estimators in zip(resolutions, neighbors, estimators))

In [ ]:
pd.read_csv("../../data/cluster_f1_scores.txt").sort_values("score", ascending=False).head(10)

### Characterize best solution

In [ ]:
res = 0.69
neighbors = 31
estimators = 226

In [ ]:
scores = Parallel(n_jobs=-1)(delayed(process_fold)(train, 
                                                       test, 
                                                       n_neighbors=neighbors,
                                                       resolution=res,
                                                       n_estimators=estimators) 
                                 for train, test in cv_folds.split(
        df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]],
        meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"],
        groups=meta.loc[epithelial_subset].loc[meta["inner"]]["fov"]
    )) # This should confirm the model selection estimates

print(np.mean(scores))

In [ ]:
# Now that we selected the model hyperparameters, retrain it on the full inner CV dataset and characterize it
# Step 1: Define metabolic clusters on training data
ad = sc.AnnData(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]])
sc.pp.neighbors(ad, n_neighbors=neighbors)
sc.tl.leiden(ad, resolution=res)
ad.obs.leiden = ad.obs.leiden.values.astype(int)

# Step 2: Define a classifier to propagate the clusters to the validation data
neigh = KNeighborsClassifier(n_neighbors=neighbors, n_jobs=-1)
neigh.fit(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].values, 
        ad.obs.leiden.values)

# Step 3: Compute proportion of cells in each cluster for each FOV in the training data
train_fov_cluster_composition = pd.DataFrame(ad.obs.leiden.values, columns=["Cluster"])
train_fov_cluster_composition["fov"] = (
    meta.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]]["fov"].values
)
train_fov_cluster_composition = (
    train_fov_cluster_composition.groupby("fov")["Cluster"].value_counts().unstack().fillna(0)
)
# Normalize by the number of cells in each FOV
train_fov_cluster_composition = (
    train_fov_cluster_composition.div(train_fov_cluster_composition.sum(axis=1), axis=0)
)

# Step 4: Train a classifier on the training data composition to predict the stage of each FOV
xgb = XGBClassifier(
    n_estimators=estimators, 
    max_depth=3, 
    device="cuda", 
    random_state=0)
xgb.fit(train_fov_cluster_composition,
        LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]].groupby("fov").first()["Stage"].loc[train_fov_cluster_composition.index]))

print("Number of metabolic clusters:", len(train_fov_cluster_composition.columns))

In [ ]:
import shap
from sklearn.metrics import confusion_matrix
explainer = shap.Explainer(xgb)
shap_values = explainer(train_fov_cluster_composition)

```
The Shapley value is the average contribution of a feature value to the prediction in different coalitions. The Shapley value is NOT the difference in prediction when we would remove the feature from the model.
```
Note: it could be more appropriate to use a custom `explainer` to rely on marginal distributions rather than conditional distributions (default for xgboost).

In [ ]:
# Purples + Green for healthy
pal_stages = [(127,201,127), (242,240,247), (203,201,226), (158,154,201), (106,81,163)]
# Convert to hex
pal_stages = ['#%02x%02x%02x' % (r, g, b) for r, g, b in pal_stages]

original_stages = meta.loc[epithelial_subset].loc[meta["inner"]].groupby("fov").first()["Stage"].loc[train_fov_cluster_composition.index]
label_encoder = LabelEncoder()
true_labels = label_encoder.fit_transform(original_stages)
predicted_labels = xgb.predict(train_fov_cluster_composition)
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print(conf_matrix)
# Few errors on training set, so not meaningful to interpret misclassified FOVs

# For each stage, see how the true positive samples were predicted (SHAP values)

# Get indices of correctly predicted samples for each class
correct_indices = {class_idx: np.where((true_labels == class_idx) & (predicted_labels == class_idx))[0] 
                   for class_idx in range(conf_matrix.shape[0])}

shap_values_list = []
for class_idx, indices in correct_indices.items():
    shap_values_class = pd.DataFrame(shap_values.values[indices, :, class_idx], columns=train_fov_cluster_composition.columns)
    shap_values_class['fov'] = train_fov_cluster_composition.index[indices]
    shap_values_class['class'] = label_encoder.inverse_transform([class_idx])[0]  # Use original stage names
    shap_values_list.append(shap_values_class)

shap_values_df = pd.concat(shap_values_list)

# Melt the DataFrame for easier plotting with plotnine
shap_values_melted = shap_values_df.melt(id_vars=['fov', 'class'], var_name='Feature', value_name='SHAP Value')

# Compute mean and std SHAP values for feature importance for each class
feature_importance = shap_values_melted.groupby(['class', 'Feature'])['SHAP Value'].agg(['mean', 'std']).reset_index()
# We want to display either high positive or high negative values
feature_importance["abs_mean"] = np.abs(feature_importance["mean"]) 
feature_importance = feature_importance.sort_values(by=['class', 'abs_mean'], ascending=[True, False])
feature_importance = feature_importance.loc[feature_importance.Feature.isin(feature_importance.groupby('class').head(2).Feature)]

# Add a column for reordering features
feature_importance['Feature'] = pd.Categorical(feature_importance['Feature'], 
                                               categories=feature_importance.groupby('class').head(2).Feature.unique()[::-1], 
                                               ordered=True)
feature_importance['class'] = feature_importance['class'].astype('category')

# Step 8: Visualize feature importance using plotnine
p2 = (ggplot(feature_importance, 
                aes(x='Feature', y='mean', fill='class'))
      + geom_bar(stat='identity', position='dodge')
      + geom_errorbar(aes(ymin='mean-std', ymax='mean+std'), width=0.2, position='dodge')
      + coord_flip()
      + theme_classic()
      + scale_fill_manual(values=pal_stages)
      + labs(title='Feature importance by stage (correctly predicted samples)', x='Metabolic cluster', y='Mean SHAP value', fill='Stage')
      + facet_wrap('~class', scales='free_y'))

ggsave(p2, "../../figures/fig3/feature_importance_correctly_predicted_samples.pdf", width=7, height=5)

p2.show()

### Interpret individual clusters
How frequent? What composition?

In [ ]:
train_fov_cluster_composition["Stage"] = "Misclassified"
for k,v in correct_indices.items():
    train_fov_cluster_composition.loc[
        train_fov_cluster_composition.index[v], 
        "Stage"] = label_encoder.inverse_transform([k])[0]
    
for clust in feature_importance.Feature.cat.categories:
    print(clust)
    # How frequent is this cluster?
    print(train_fov_cluster_composition[clust].mean())

# Step 1: Compute the average frequency of each cluster for each stage
average_frequency = train_fov_cluster_composition.groupby("Stage").mean().reset_index()

# Melt the DataFrame for easier plotting with plotnine
average_frequency_melted = average_frequency.melt(id_vars=['Stage'], var_name='Cluster', value_name='Frequency')
average_frequency_melted = average_frequency_melted.loc[average_frequency_melted.Stage != "Misclassified"]
average_frequency_melted = average_frequency_melted.loc[average_frequency_melted["Cluster"].isin(feature_importance.Feature.cat.categories)]

# Step 2: Prepare the data for plotting
# Ensure that the 'Stage' column is treated as a categorical variable
average_frequency_melted['Stage'] = pd.Categorical(average_frequency_melted['Stage'], 
                                                   categories=average_frequency_melted['Stage'].unique(), 
                                                   ordered=True)
average_frequency_melted['Cluster'] = pd.Categorical(average_frequency_melted['Cluster'], 
                                                     categories=feature_importance.Feature.cat.categories, 
                                                     ordered=True)

# Step 3: Create the dot plot using plotnine
p = (ggplot(average_frequency_melted, aes(x='Cluster', y='Stage', size='Frequency'))
     + geom_point()
     + theme_classic()
     + coord_flip()
     + scale_size_continuous(range=[2, 15])
     + labs(title='Average frequency of clusters by sample', x='Cluster', y='Stage', size='Average frequency'))

ggsave(p, "../../figures/fig3/average_frequency_clusters_by_stage.pdf", width=7, height=5)

p.show()

In [ ]:
# Step 1: Filter the Data
shortlisted_features = feature_importance.Feature.cat.categories
shortlisted_indices = ad.obs.leiden.isin(shortlisted_features)

# Step 2: Group by Cluster
# Create a DataFrame with marker expression and cluster information
marker_expression = pd.DataFrame(ad.X[shortlisted_indices, :], columns=ad.var_names)
marker_expression['Cluster'] = ad.obs['leiden'].loc[shortlisted_indices].values

# Group by cluster and compute the mean expression for each marker
grouped_expression = marker_expression.groupby('Cluster').mean()

# Step 3: Cluster the Result
# Perform hierarchical clustering on the grouped data

# Perform hierarchical clustering
row_linkage = linkage(grouped_expression, method='complete')
col_linkage = linkage(grouped_expression.T, method='complete')

# Get the order of rows and columns
row_order = leaves_list(row_linkage)
col_order = leaves_list(col_linkage)

# Scale df values so that each column (abundance for a given marker) is between 0 and 1
grouped_expression = grouped_expression.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)), axis = 0)

# Reorder the DataFrame
df_reordered = grouped_expression.iloc[row_order, :].iloc[:, col_order]

# Melt the DataFrame for plotnine
df_melted = df_reordered.melt(ignore_index=False).reset_index()

# Update the levels of the categorical variables to reflect the new order
df_melted["Cluster"] = pd.Categorical(df_melted["Cluster"], categories=df_reordered.index, ordered=True)
df_melted["variable"] = pd.Categorical(df_melted["variable"], categories=df_reordered.columns.to_list(), ordered=True)

In [ ]:
# Display as heatmap
gp = (
    ggplot(df_melted, aes(x="variable", y="Cluster")) 
    + geom_tile(aes(fill="value")) 
    + theme_classic() 
    + theme(axis_text_x=element_text(angle=90)) 
    + labs(y="Epithelial cluster", x="Marker", fill="Median\nabundance") 
    # Use reverted plasma color palette
    + scale_fill_gradientn(colors = ["#EFF822", "#CC4977","#0F0782"])
)
print(gp)
ggsave(gp, f"../../figures/fig3/heatmap_metab_markers_clusters.png", width = 6, height = 3.2)
ggsave(gp, f"../../figures/fig3/heatmap_metab_markers_clusters.pdf", width = 6, height = 3.2)

```python
from sklearn.cluster import HDBSCAN

hdb = HDBSCAN(min_cluster_size = 25)
hdb.fit(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].iloc[train].values)
pd.Series(hdb.labels_).value_counts()
```

Note: maybe the computation could be made faster if the KNN classifier used the precomputed NN from scanpy? In practice it is not faster (as scanpy's distance computation is slower than scipy's).
```python
from scipy.spatial import distance_matrix

for train, test in cv_folds.split(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]],
                                  meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"],
                                  groups=meta.loc[epithelial_subset].loc[meta["inner"]]["fov"]):
    n_neighbors = 30

    # Step 1: Define metabolic clusters on training data
    ad = sc.AnnData(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].iloc[train])
    sc.pp.neighbors(ad, n_neighbors=n_neighbors)
    sc.tl.leiden(ad, resolution=0.5)
    ad.obs.leiden = ad.obs.leiden.values.astype(int)

    # Step 2: Define a classifier to propagate the clusters to the test data
    neighbors_graph = ad.obsp['connectivities'] # Extract nn graph
    neighbors_graph = sort_graph_by_row_values(neighbors_graph)
    neigh = KNeighborsClassifier(n_neighbors=n_neighbors, metric = 'precomputed')
    neigh.fit(neighbors_graph, 
              ad.obs.leiden.values)

    # Step 3: Compute proportion of cells in each cluster for each FOV in the training data
    train_fov_cluster_composition = pd.DataFrame(ad.obs.leiden.values, columns=["Cluster"])
    train_fov_cluster_composition["fov"] = (
        meta.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].iloc[train]["fov"].values
    )
    train_fov_cluster_composition = (
        train_fov_cluster_composition.groupby("fov")["Cluster"].value_counts().unstack().fillna(0)
    )
    # Normalize by the number of cells in each FOV
    train_fov_cluster_composition = (
        train_fov_cluster_composition.div(train_fov_cluster_composition.sum(axis=1), axis=0)
    )

    # Step 4: Train a classifier on the training data composition to predict the stage of each FOV
    xgb = XGBClassifier(
        n_estimators=250, 
        max_depth=3, 
        device="cuda", 
        random_state=0)
    xgb.fit(train_fov_cluster_composition,
            LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]].groupby("fov").first()["Stage"].loc[train_fov_cluster_composition.index]))
    
    # Step 5: Predict clusters on test data
    test_distances = distance_matrix(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].iloc[test].values,
                                     df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].iloc[train].values)
    test_clusters = neigh.predict(test_distances)
    
    # Step 6: Compute proportion of cells in each cluster for each FOV in the test data
    test_fov_cluster_composition = pd.DataFrame(test_clusters, columns=["Cluster"])
    test_fov_cluster_composition["fov"] = meta.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]].iloc[test]["fov"].values
    test_fov_cluster_composition = test_fov_cluster_composition.groupby("fov")["Cluster"].value_counts().unstack().fillna(0)
    # Normalize by the number of cells in each FOV
    test_fov_cluster_composition = test_fov_cluster_composition.div(test_fov_cluster_composition.sum(axis=1), axis=0)
    
    # Ensure all clusters are covered in the testing dataframe
    for cluster in train_fov_cluster_composition.columns:
        if cluster not in test_fov_cluster_composition.columns:
            test_fov_cluster_composition[cluster] = 0

    # Reorder columns to match the training set
    test_fov_cluster_composition = test_fov_cluster_composition[train_fov_cluster_composition.columns]

    # Step 7: Predict stage of each FOV in the test data
    preds = xgb.predict(test_fov_cluster_composition)

    # Step 8: Compute f1_score
    print(f1_score(LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]].groupby("fov").first()["Stage"].loc[test_fov_cluster_composition.index]), 
                   preds, 
                   average="macro"))
```

### Score 1.1: Less markers
Match the marker used to later derive pathway scores

In [ ]:
subset_markers = ['CD98', 'CytC', 'MCT1', 'ASCT2', 'LDH', 'GS', 'GLS', 'ATP5A',
       'CS', 'PKM2', 'GLUT1']


cross_val_score(
    XGBClassifier(
        n_estimators=250, 
        max_depth=3, 
        device="cuda", 
        random_state=0),
    df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"], subset_markers],
    LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"]),
    groups=meta.loc[epithelial_subset].loc[meta["inner"]]["fov"], 
    cv=cv_folds, 
    scoring='f1_macro')

### Score 1.2: More markers
Use all available measurements, including lineage markers and morphology.

In [ ]:
df_all = pd.DataFrame(
    np.hstack([
        adata.obs.loc[adata.obs.consensus == "Cancer_cell", adata.obs.columns[:32].drop(['centroid-0','centroid-1'])].values,
        adata.X[adata.obs.consensus == "Cancer_cell",:]
    ]),
    columns= adata.obs.columns[:32].drop(['centroid-0','centroid-1']).to_list() + adata.var_names.to_list()
)
df_all.index = df.index

In [ ]:
cross_val_score(
    XGBClassifier(
        n_estimators=250, 
        max_depth=3, 
        device="cuda", 
        random_state=0),
    df_all.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]],
    LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"]),
    groups=meta.loc[epithelial_subset].loc[meta["inner"]]["fov"], 
    cv=cv_folds, 
    scoring='f1_macro')

### Score 2: Unweighted average per main pathway

In [ ]:
main_pathways = {
    "Glycolysis and Gluconeogenesis" : ["MCT1", "PKM2", "LDH", "GLUT1"],
    "Oxidative phosphorylation" : ["ATP5A", "CytC"],
    "TCA cycle" : ["CS", "MCT1"],
    "Glutamate metabolism" : ["GLS", "GS", "CD98", "ASCT2"],
    "Valine, Leucine and Isoleucine Metabolism": ["ASCT2", "MCT1"]
}

In [ ]:
def comp_score(v):
    # Get corresponding values and z-score per column
    w = df.loc[:,v].apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)
    
    return np.mean(w, axis = 1)

df_metab = pd.DataFrame({k: comp_score(v) for k,v in main_pathways.items()})


In [ ]:
cross_val_score(
    XGBClassifier(
        n_estimators=250, 
        max_depth=3, 
        device="cuda", 
        random_state=0),
    df_metab.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]],
    LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"]),
    groups=meta.loc[epithelial_subset].loc[meta["inner"]]["fov"], 
    cv=cv_folds, 
    scoring='f1_macro')

### Score 3: Weighted pathway averages

In [ ]:
weighted_pathways = {
    "Glycolysis and Gluconeogenesis" : {"MCT1":0.2, "PKM2": 5, "LDH": 1, "GLUT1": 0.2},
    "Oxidative phosphorylation" : {"ATP5A": 1, "CytC": 1},
    "TCA cycle" : {"CS": 5, "MCT1": 0.2},
    "Glutamate metabolism" : {"GLS": 1, "GS": 1, "CD98": 0.2, "ASCT2": 0.2},
}

In [ ]:
def comp_score(v, k):
    # Get corresponding values and z-score per column
    w = df.loc[:,v.keys()].apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)
    
    # Computed weighted mean based on coefficient of each marker
    weights = weighted_pathways[k]
    w = w.apply(lambda x: x*weights[x.name], axis = 0)
    return(np.sum(w, axis = 1)/sum(weights.values()))

df_metab2 = pd.DataFrame({k: comp_score(v,k) for k,v in weighted_pathways.items()})

In [ ]:
cross_val_score(
    XGBClassifier(
        n_estimators=250, 
        max_depth=3, 
        device="cuda", 
        random_state=0),
    df_metab2.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]],
    LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"]),
    groups=meta.loc[epithelial_subset].loc[meta["inner"]]["fov"], 
    cv=cv_folds, 
    scoring='f1_macro')

### Score 4: network diffusion of abundance within pathways

#### Step 4.1: get curated pathway membership
Curated list obtained from the supplementary material of [Gaude & Frezza (2016)](https://doi.org/10.1038/ncomms13041). Supplementary Data 3 was downloaded and saved in CSV format.

In [ ]:
curated_pw = pd.read_csv("../../data/41467_2016_BFncomms13041_MOESM340_ESM.csv", header=1)
# Rename for consistency
main_pathways["Citric Acid Cycle"] = main_pathways["TCA cycle"]
main_pathways['Oxidative Phosphorylation'] = main_pathways['Oxidative phosphorylation']
# Get the gene symbols for the main pathways
all_metab_gene_symbols = curated_pw.loc[curated_pw.Pathways.isin(main_pathways.keys()),"Genes"].unique()

#### Step 4.2: get co-expression network of relevant metabolic genes

Single-cell atlas obtained from the HTAN [Chen, Scurrah et al study (2021)](https://doi.org/10.1016/j.cell.2021.11.031). The "Discovery (DIS) set of human colorectal tumor: Epithelial" dataset was downloaded in H5AD format from the [Cell x Gene platform](https://cellxgene.cziscience.com/collections/a48f5033-3438-4550-8574-cdff3263fdfd) on 2024-07-24.

In [ ]:
import networkx as nx

In [ ]:
atlas = ad.read_h5ad('../../data/5bc8d9bb-f6a4-4f7e-add7-2825cbd36625.h5ad')
# Replace for concistency with curated pathways
atlas.var.feature_name.replace({"PKM":"PKM2"}, inplace=True)
# Keep only the metabolic markers
atlas = atlas[:,atlas.var.feature_name.isin(all_metab_gene_symbols)]

In [ ]:
# Compute pearson correlation between genes (columns)
corrmat = atlas.to_df().corr()
corrlist = corrmat.melt(ignore_index=False).reset_index()
corrlist = corrlist.loc[corrlist["index"] < corrlist["variable"],:]

In [ ]:
# We drop low correlations to make the network sparser, but aim to keep at least 80% of the markers in the network
corr_thr = 0.03
cfilt = corrlist.loc[(corrlist["value"] > corr_thr)|(corrlist["value"] < -corr_thr),:]
g = nx.from_pandas_edgelist(cfilt, source="index", target="variable", edge_attr=True)
assert nx.number_of_nodes(g) >= 0.8 * atlas.n_vars

#### Step 4.3: Network diffusion of marker weights

In [ ]:
from scipy.sparse.linalg import cg  # Conjugate Gradient solver
from scipy.sparse import eye, diags

def random_walk_with_restart_corrected(g, node_values, r=0.1, precomputed_T=None):
    # Ensure node_values is a numpy array
    if not isinstance(node_values, np.ndarray):
        node_values = np.array(list(node_values.values()))
    
    # Create a transition probability matrix from the adjacency matrix
    # Use precomputed transition matrix if available
    if precomputed_T is not None:
        T = precomputed_T
    else:
        A = nx.adjacency_matrix(g)
        row_sums = np.array(A.sum(axis=1)).flatten()
        row_sums[row_sums == 0] = 1  # Avoid division by zero for isolated nodes
        T = A.multiply(1 / row_sums[:, np.newaxis])
    
    # Identity matrix
    I = eye(T.shape[0])
    
    # Linear system: (I - (1 - r) * T) x = r * v
    A = I - (1 - r) * T
    b = r * node_values
    
    # Solve the linear system using Conjugate Gradient method
    x, _ = cg(A, b, tol=1e-8, maxiter=1000)
    
    return x

Note: even with attempts at parallelization, the following cell takes ~n hours (>100 minutes) to run.

In [ ]:
from joblib import Parallel, delayed

# Association between pathway genes and markers
gene_to_marker_dict = {
    'SLC3A2': 'CD98',
    'CYC1': 'CytC',
    'SLC16A1': 'MCT1',
    'SLC1A5': 'ASCT2',
    'LDHA': 'LDH',
    'LDHB': 'LDH',
    'GLUL': 'GS',
    'GLS': 'GLS',
    'ATP5A1': 'ATP5A',
    'CS': 'CS',
    'PKM2': 'PKM2',
    'SLC2A1': 'GLUT1',
}
# Vectorize operations instead of using apply
df_ini = adata.obs.loc[adata.obs.consensus == "Cancer_cell", metab_markers]
df_ini = (df_ini - df_ini.mean())/df_ini.std()
# For testing only
df_ini = df_ini.sample(30000, random_state=0)

symb_dict = atlas.var.feature_name.to_dict()
index_to_marker = {i: gene_to_marker_dict[symb_dict[gene_ensg]]
                   for i, gene_ensg in enumerate(g.nodes)
                   if symb_dict[gene_ensg] in gene_to_marker_dict}

def diffused_abundance(g, seed_values, index_to_marker, r=0.1, precomputed_T=None):
    initial_values = np.zeros(len(g.nodes))  # Use numpy for initialization
    for i, m in index_to_marker.items():
        initial_values[i] = seed_values[m]
    
    final_values_array = random_walk_with_restart_corrected(g, initial_values, r=r, precomputed_T=precomputed_T)
    return final_values_array

# Precompute RWR transition matrix
A = nx.adjacency_matrix(g)
row_sums = np.array(A.sum(axis=1)).flatten()
row_sums[row_sums == 0] = 1  # Avoid division by zero for isolated nodes
T = A.multiply(1 / row_sums[:, np.newaxis])

# Parallel processing to speed up diffused_abundance computation
# Step 1: Generate list of tuples using Parallel processing
results = Parallel(n_jobs=-1)(delayed(lambda i: (df_ini.index[i], diffused_abundance(g=g, seed_values=df_ini.iloc[i], 
                                                                                     index_to_marker=index_to_marker,
                                                                                     precomputed_T=T)))(i) for i in range(df_ini.shape[0]))

# Step 2: Convert list of tuples into a dictionary
results_dict = dict(results)

# Step 3: Create DataFrame from the dictionary
df_diff = pd.DataFrame.from_dict(results_dict, orient="index", columns=[symb_dict[n] for n in g.nodes()])

In [ ]:
df_diff = pd.read_csv("../../data/diffused_abundance.csv", index_col=0) # If pre-computed
df_diff.index = df_diff.index.astype(str)
# df_diff.to_csv("diffused_abundance.csv")

#### Step 4.4: Compute weighted score from diffused abundance values

In [ ]:
curated_pw_dict = {p : {} for p in curated_pw.loc[curated_pw.Pathways.isin(main_pathways.keys()),"Pathways"].unique()}
# Convert data frame of genes and pathways to a dictionary of genes in each pathway
for gene, path in curated_pw.loc[curated_pw.Pathways.isin(main_pathways.keys()),:].values:
    if gene in df_diff.columns:
        curated_pw_dict[path][gene] = 1.0 # Each gene has a pathway-specific weight, default to 1.0

In [ ]:
def comp_score(v, k):
    # Get corresponding values and z-score per column
    pathway_abundances = df_diff.loc[:,v.keys()]
    
    # Computed weighted mean based on coefficient of each marker
    pathway_abundances = pathway_abundances.apply(lambda x: x*v[x.name], axis = 0)
    return(np.sum(pathway_abundances, axis = 1)/sum(v.values()))

df_metab3 = pd.DataFrame({k: comp_score(v,k) for k,v in curated_pw_dict.items()})

In [ ]:
cross_val_score(
    XGBClassifier(
        n_estimators=250, 
        max_depth=3, 
        device="cuda", 
        random_state=0),
    df_metab3.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]],
    LabelEncoder().fit_transform(meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"]),
    groups=meta.loc[epithelial_subset].loc[meta["inner"]]["fov"], 
    cv=cv_folds, 
    scoring='f1_macro')

### Score 5: Diffusion of all available markers

In [ ]:
# No marker can be readily added to the diffusion process without rebuilding the network
print([x for x in adata.var_names if x in symb_dict.values()])
print([x for x in adata.obs.columns[:23] if x in symb_dict.values()])

If we want to explore further the potential of gene network diffusion to complement the raw abundance values, the following steps could be followed:  
* Create complete correlation network from atlas
* Convert to cost ($1 - |\rho |$)
* Compute shortest path between seed nodes and corresponding betweeness centrality per edge
* Delete all null-weight edges and any resulting connected component that does not include any seed node. All remaining connected component should have at least two seed nodes
* Use backboning / thresholding in each connected component
* Do diffusion of abundance from seed nodes
* Use all diffused node values as input to an XGBoost model

The following resource could be explored to study protein-metabolite associations:

```R
library(cosmosR)
data(meta_network)
```

```
#' @format An object of class \dQuote{\code{tibble}} with 117065 rows
#'   (interactions) and three variables:
#'   \describe{
#'     \item{\code{source}}{Source node, either metabolite or protein}
#'     \item{\code{interaction}}{Type of interaction, 1 = Activation, -1 = Inhibition}
#'     \item{\code{target}}{Target node, either metabolite or protein}
#'   A detailed description of the identifier formatting can be found under 
#'   \url{https://metapkn.omnipathdb.org/00__README.txt}.
#'   }
```

**Glycolysis**: GLUT1, PKM2, LDH, MCT1  
**Fatty acid oxidation**: CPT1A  
**TCA cycle**: CS  
**Amino acid metabolism**: ARG1, CD98, GLS, GS, ASCT2  
**OxPhos**: CytC, ATP5A  
**Cancer and proliferation**:CA9, Ki67  

In curated list:  
**Glycolysis and Gluconeogenesis**: PKM2, LDH (LDHA, LDHB), GLUT1 (SLC2A1), MCT1 (SLC16A1)  
**Carnitine shuttle**: CPT1A  
**Citric Acid Cycle**: CS, MCT1  
**Urea Cycle**: ARG1  
**Glutamate metabolism**: CD98 (SLC3A2), GLS, GS (GLUL), ASCT2 (SLC1A5)  
**Transport, Extracellular**: CD98  
**Glycine, Serine and Threonine Metabolism**: ASCT2  
**Valine, Leucine and Isoleucine Metabolism**: ASCT2, MCT1  
**Alanine and Aspartate Metabolism**: ASCT2  
**Cysteine Metabolism**: ASCT2  
**Oxidative Phosphorylation**: CytC (CYC1?), ATP5A (ATP5A1?)  
**Carbonic Acid Metabolism**: CA9  
**Transport, Golgi Apparatus**: GLUT1  
**Biotin Metabolism**: MCT1  
**Ketone Bodies Metabolism**: MCT1  
**Propanoate Metabolism**: MCT1  

Missing Ki67. 



## 